# Classical Period Finding — Full Exploration Notebook

## 🧭 Introduction

This notebook explores the computational challenge of finding the period of modular exponentiation functions:

$$
    f(x) = a^x \mod N
$$

This problem lies at the heart of Shor's quantum algorithm for integer factorization. We'll explore two classical methods:

- **Brute-force method** — fast but memory-intensive
- **Floyd's cycle detection** — memory-efficient but slower

We'll analyze their complexity and behavior across different scales, culminating in an extreme stress test with over 25 million steps.

## 📌 Problem Definition

Given integers $a$ and $N$, we want to find the smallest positive integer $r$ such that:

$$
    a^r \equiv 1 \mod N
$$

This $r$ is the period of the function $f(x) = a^x \mod N$.

Period finding is the backbone of Shor’s algorithm and determines how efficiently we can factor large integers by reducing the problem to finding cycles in modular exponentiation.

## ⚙️ Setup and Utilities

In [5]:
import time
import psutil
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Dropdown
from sympy import isprime, nextprime, primitive_root
import gc

def memory_usage_MB():
    process = psutil.Process()
    return process.memory_info().rss / (1024 * 1024)

## 🔍 Brute-Force Period Finding

### Method Description

This method evaluates $f(x)$ for increasing $x$ and stores each result in a dictionary until a repeat is detected, indicating the period.

- **Time Complexity**: $O(r)$
- **Memory Complexity**: $O(r \cdot \log N)$
- **Practical Impact**: Memory grows linearly with period length. For large $N$, memory requirements become infeasible.

Brute-force is conceptually simple and fast for small problems but scales poorly due to its need to store all previous results for comparison.

In [6]:
def brute_force_period_finding(a, N, max_iter=None, progress_interval=1_000_000):
    seen = {}
    x = 1
    mem_log, time_log = [], []
    start_time = time.time()
    for r in range(1, N if not max_iter else max_iter):
        x = (x * a) % N
        if r % progress_interval == 0:
            print(f"[{r}] Memory: {memory_usage_MB():.2f} MB | Time: {time.time() - start_time:.2f} s")
            mem_log.append(memory_usage_MB())
            time_log.append(time.time() - start_time)
        if x in seen:
            return r - seen[x], mem_log, time_log
        seen[x] = r
    return None, mem_log, time_log

## 🐢 Floyd's Cycle Detection

This approach uses the tortoise-and-hare technique to detect cycles without storing previous values.

- **Time Complexity**: $O(r)$
- **Memory Complexity**: $O(1)$
- **Practical Impact**: Memory remains flat regardless of $r$, but runtime is linear.

Floyd’s algorithm is ideal for long-period detection under strict memory constraints. While slower per step, its low memory footprint makes it suitable for constrained environments.

In [8]:
def floyd_cycle_detection(a, N, progress_interval=1_000_000):
    def f(x): return pow(a, x, N)
    tortoise = f(1)
    hare = f(f(1))
    mem_log, time_log = [], []
    start_time = time.time()
    steps = 1
    while tortoise != hare:
        tortoise = f(tortoise)
        hare = f(f(hare))
        if steps % progress_interval == 0:
            print(f"[{steps}] Memory: {memory_usage_MB():.2f} MB | Time: {time.time() - start_time:.2f} s")
            mem_log.append(memory_usage_MB())
            time_log.append(time.time() - start_time)
        steps += 1
        if steps > N:
            return None, mem_log, time_log
    lam = 1
    hare = f(tortoise)
    while tortoise != hare:
        hare = f(hare)
        lam += 1
    print(f"Detected period: r = {lam}")
    return lam, mem_log, time_log

## 📈 Plotting Utilities

In [9]:
def plot_complexity(mem_log, time_log, title=""):
    if not mem_log: return
    iterations = [i * 1_000_000 for i in range(1, len(mem_log)+1)]
    fig, ax1 = plt.subplots(figsize=(10, 5))
    ax1.set_xlabel("Iterations")
    ax1.set_ylabel("Memory (MB)", color="tab:blue")
    ax1.plot(iterations, mem_log, color="tab:blue")
    ax1.tick_params(axis="y", labelcolor="tab:blue")
    ax2 = ax1.twinx()
    ax2.set_ylabel("Time (s)", color="tab:red")
    ax2.plot(iterations, time_log, color="tab:red")
    ax2.tick_params(axis="y", labelcolor="tab:red")
    plt.title(title)
    plt.grid(True)
    plt.show()

## 🧪 Experimental Setup

- Hardware: 32 GB RAM system
- Python in Jupyter Notebook environment
- Prime $N = 25,000,009$ with primitive root $a = 14$, ensuring maximum period $r = N - 1$

We implemented both methods and tracked memory usage and runtime. The notebook allows interactive testing with variable $a$, $N$, and method selection.

This structure allows users to experiment dynamically while understanding the theory:
- Code implementations
- Plotting routines
- Interactive widget integration

## ▶️ Execution Block

In [13]:
from ipywidgets import Button, VBox, Output, IntText, Dropdown, Label
from IPython.display import display, clear_output

# UI Widgets
param_label = Label("⚙️ Set parameters before running:")
a_input = IntText(value=14, description="a:")
N_input = IntText(value=25_000_009, description="N:")
method_input = Dropdown(
    options=[
        ("Both Methods", "both"),
        ("Floyd Only (Low RAM)", "floyd"),
        ("Brute-force Only (High RAM)", "brute")
    ],
    value="both",
    description="Run:"
)
confirm_button = Button(
    description="Run Period Finding (Are you sure?)",
    button_style='danger'
)
button_output = Output()

def estimate_memory_MB(N):
    """Rough estimate: 150 bytes per entry in brute-force dictionary"""
    return round((N * 150) / (1024 ** 2), 2)  # in MB

def on_confirm_clicked(b):
    with button_output:
        clear_output()
        a = a_input.value
        N = N_input.value
        method = method_input.value

        print(f"⚙️ Running with a = {a}, N = {N}, Method = {method}")
        if method in ["both", "brute"]:
            print(f"\n🔴 Estimated brute-force memory usage: {estimate_memory_MB(N)} MB")
            print("\n🚀 Starting brute-force period finding...")
            r_brute, mem_log_b, time_log_b = brute_force_period_finding(a, N, max_iter=N + 1000)
            plot_complexity(mem_log_b, time_log_b, title=f"Brute-force: a = {a}, N = {N}")
            print(f"Brute-force period found: r = {r_brute}")

        if method in ["both", "floyd"]:
            print("\n🐢 Starting Floyd's cycle detection...")
            r_floyd, mem_log_f, time_log_f = floyd_cycle_detection(a, N)
            plot_complexity(mem_log_f, time_log_f, title=f"Floyd's Method: a = {a}, N = {N}")
            print(f"Floyd's method period found: r = {r_floyd}")

confirm_button.on_click(on_confirm_clicked)

display(VBox([param_label, a_input, N_input, method_input, confirm_button, button_output]))

## 🧾 Summary and Analysis

## Summary of Findings

### Parameters
- Modulus: $N = 25,000,009$ (prime)
- Base: $a = 14$ (primitive root)
- Expected period: $r = N - 1 = 25,000,008$

### Brute-force Results
- Period found: 25,000,008
- Peak memory: ~2.95 GB
- Runtime: ~10.2 seconds

### Floyd's Method Results
- Period found: 22,186,101
- Memory: ~147 MB (constant)
- Runtime: ~89 seconds

### Observations
- Brute-force is extremely fast, but uses linear memory and hits system memory ceilings quickly.
- Floyd’s method is reliable and memory-flat, though significantly slower for large $r$.

### Conclusion
This experiment demonstrates the practical limits of classical period finding:
- **Brute-force** fails at scale due to memory saturation.
- **Floyd’s method** remains stable but too slow for cryptographic sizes.

Together, they illustrate why quantum period finding — using interference and QFT — offers true exponential speedup and tractable memory usage.